In [2]:
import boto3
import pandas as pd
import awswrangler as wr

In [3]:
my_session = boto3.session.Session(profile_name='099132402094_Data-Science-Prod-Custom-Access')
ath = my_session.client(service_name = 'athena',
                      region_name = 'us-west-2')

In [4]:
df_birthday = pd.read_csv('/Users/eviemcgonigle/Desktop/generation_channels - chanDOB.csv')

In [5]:
post_uploaded_cutoff = "2023-08-02"
df_written = wr.athena.read_sql_query(
    sql = f"""
    SELECT DISTINCT
            C.CHANNEL_ID,
            C.CHANNEL_ABOUT,
            C.CHANNEL_TITLE,
            C.CHANNEL_NAME,
            PL.POST_TEXT,
            PL.POST_TITLE,
            PL.POST_ID
        FROM datalake_silver_catalog.channel_latest C
        JOIN datalake_silver_catalog.post_latest PL ON C.CHANNEL_ID = PL.CHANNEL_ID
        WHERE C.CHANNEL_ID IN {tuple(df_birthday["chan_id"])}
            AND PL.POST_UPLOADED >= '{post_uploaded_cutoff}'
    """,
    database="datalake_silver_catalog",
    boto3_session = my_session
)
df_written.head()

,channel_id,channel_about,channel_title,channel_name,post_text,post_title,post_id
0,fecb0a066814392d8ce17c238a99391d,Marcel Irnie - Pro Superbike Racer & Rider Tra...,Marcel Irnie,irnieracing,Ontario #carbontax #dougford #farmers.,Ontario Farmers & Doug Ford Oppose TRUDEAU CAR...,7df5a31ac0ef3accb5903f985285a97b
1,fecb0a066814392d8ce17c238a99391d,Marcel Irnie - Pro Superbike Racer & Rider Tra...,Marcel Irnie,irnieracing,Opinion: Wanted — a federal leader who will be...,TRUDEAU & POILIEVRE: NOT honest about CLIMATE ...,3567986487a03dc9b9bb637f8d118f7a
2,fecb0a066814392d8ce17c238a99391d,Marcel Irnie - Pro Superbike Racer & Rider Tra...,Marcel Irnie,irnieracing,N/A,Trump back on Twitter X with 86 Million Follow...,291999acacfc32caa5b25996de8d170d
3,fecb0a066814392d8ce17c238a99391d,Marcel Irnie - Pro Superbike Racer & Rider Tra...,Marcel Irnie,irnieracing,N/A,RIDE OR DIE | IrnieracingNews Digital Nomad,c8881bb9be5f36ec812104f628aa8160
4,fecb0a066814392d8ce17c238a99391d,Marcel Irnie - Pro Superbike Racer & Rider Tra...,Marcel Irnie,irnieracing,N/A,A Massive Defeat For Ukraine / NATO,23d9f2793d433f599b7380e3657d87a6


In [6]:
df_written = df_written.rename(columns={'channel_id': 'chan_id'})
df_merged = df_birthday.merge(df_written, on='chan_id')
df_merged = df_merged.drop_duplicates(subset=['chan_id', 'post_id'])

In [45]:
df_merged.to_csv('/Users/eviemcgonigle/Desktop/posts_gen_channels.csv')

In [24]:

# Group by 'channel' and concatenate 'post_text' and 'post_title'
def consolidate_posts(group):
    combined = " ".join(group['post_text'] + " " + group['post_title'])
    return pd.Series({'text': combined})

# Define the mapping from age_group to label
age_label = {
    'preMil': 2,
    'millennial': 1,
    'genZ': 0
}


In [26]:
#df_limited = df_merged.groupby('chan_id').head(10)
df_limited = pd.read_csv('/Users/eviemcgonigle/Desktop/distinctpost10_genchannels.csv')

In [27]:
df_limited['post_text'] = df_limited['post_text'].astype(str).fillna('')
df_limited['post_title'] = df_limited['post_title'].astype(str).fillna('')

# convert generations to labels
df_limited['label'] = df_limited['age_group'].map(age_label)
df_limited= df_limited.dropna(subset=['label']).reset_index(drop=True)

# consolidate text
df_consolidated = df_limited.groupby(['label', 'channel', 'channel_about']).apply(consolidate_posts).reset_index()

df_consolidated.head()

# Map age_group to label and filter out unwanted channels
#df_consolidated['label'] = df_consolidated['age_group'].map(age_group_to_label)
#df_consolidated = df_consolidated.dropna(subset=['label']).reset_index(drop=True)


,label,channel,channel_about,text
0,0.0,%C4%8E%C3%BCrkR%C3%A3d%E6%84%9BF%C5%91r%C3%A2P...,"Hello, welcome to my channel, here on this cha...",nan Why I Hate The CP Term nan To Anyone Who's...
1,0.0,0wenfox,Being in a relationship will bring you face to...,"nan Weekly Live Q&A: Water/Dry Fasting, Bounda..."
2,0.0,10kadam277,Welcome to my YouTube Channel in This Channel ...,#meme​ #memes #minecraft​\nhmmmmm\n\n#minecraf...
3,0.0,197Media,Southern California music documentation.\nNew ...,nan (197 Media) Fatal Wounds - Live at For the...
4,0.0,1araquinn,she/they\nbusiness 💌 nina@grail-talent.com\n🇵🇸...,nan little bday haul nan delicious now we liv...


In [28]:
df_consolidated.to_csv('generation3_lcat.csv')